In [2]:
# 粗粒度匹配 （知识点语义）



# 细粒度匹配 （题目和具体文本）



In [62]:
import os
from langchain_neo4j import Neo4jGraph

os.environ["NEO4J_URI"] = "neo4j+s://5f494a0d.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "JPvH9AyoJVJDlbOzUml3qfyA_oWutzcMfQCJI9qRCOY"

graph = Neo4jGraph()

In [56]:
assign_id_query = """
MATCH (kp:章) // 匹配所有知识点结点
WITH kp ORDER BY id(kp) // 先按 Neo4j 内部 ID 排序
WITH collect(kp) AS kps  // 收集所有知识点结点
UNWIND range(0, size(kps) - 1) AS i
WITH kps[i] AS kp, 2000 + i AS newId
SET kp.id = newId
"""
graph.query(assign_id_query)

[]

In [58]:
assign_id_query = """
MATCH (kp:节) // 匹配所有知识点结点
WITH kp ORDER BY id(kp) // 先按 Neo4j 内部 ID 排序
WITH collect(kp) AS kps  // 收集所有知识点结点
UNWIND range(0, size(kps) - 1) AS i
WITH kps[i] AS kp, 3000 + i AS newId
SET kp.id = newId
"""
graph.query(assign_id_query)

[]

In [60]:
import pandas as pd
import re
query = """
MATCH (n)
WHERE n:`章` OR n:`节`
RETURN n.id AS id, n.title AS name
"""
result = graph.query(query)
chinese_digits = {
    "零": 0, "一": 1, "二": 2, "三": 3, "四": 4, "五": 5, "六": 6, "七": 7, "八": 8, "九": 9,
    "十": 10, "百": 100, "千": 1000
}

def chinese_to_number(chinese):
    result = 0
    temp = 0
    for char in chinese:
        if char in chinese_digits:
            temp += chinese_digits[char]
        elif char == '十' and temp == 0:
            temp = 10
        elif char == '十' and temp != 0:
            temp *= 10
        elif char in ['百', '千']:
            result += temp * chinese_digits[char]
            temp = 0
    return result + temp

# 处理结果并拆分 name 字段
def split_name(name):
    # 提取汉字数字并转换为阿拉伯数字
    match = re.match(r"(\D+)(\D+)(章|节)\s*(.*)", name)
    if match:
        chinese_number = match.group(2)
        number = chinese_to_number(chinese_number)  # 将汉字数字转换为阿拉伯数字
        name_part = match.group(4).strip()  # 后面的名称部分
        return str(number), name_part
    else:
        return None, name  # 如果格式不匹配，直接返回原名称

# 遍历查询结果，拆分 name
split_data = []
for record in result:
    try:
        chapter_or_section, name_part = split_name(record['name'])
        split_data.append({
            "id": record['id'],
            "chapter_or_section": chapter_or_section,
            "name": name_part
        })
    except Exception as e:
        print(f"Error processing record {record}: {e}")
        split_data.append({
            "id": record['id'],
            "chapter_or_section": None,
            "name": record['name']
        })

# 创建 DataFrame 来查看拆分后的数据
df = pd.DataFrame(split_data)

# 输出拆分后的结果
print(df)

      id chapter_or_section          name
0   2000                  1        教育与教育学
1   3000                  1          教育概述
2   3001                  2         教育学概述
3   2001                  2         教育与发展
4   3002                  1      教育的产生与发展
5   3003                  2       教育与人的发展
6   3004                  3       教育与社会发展
7   3005                  4     全民教育与终身学习
8   2002                  3          教育目的
9   3006                  1    教育目的的意义与性质
10  3007                  2     教育目的的价值取向
11  3008                  3     中国教育目的的发展
12  2003                  4     教育制度与教育政策
13  3009                  1        教育制度概述
14  3010                  2        中国教育制度
15  3011                  3        中国教育政策
16  2004                  5        学生及其发展
17  3012                  1            学生
18  3013                  2       学生发展与教育
19  2005                  6            教师
20  3014                  1        教师职业概述
21  3015                  2  教师的专业素养与专业发展
22  3016                  3  教师学习共

In [78]:
chapter_or_section = df[['id', 'name']].to_dict(orient='records')
print(chapter_or_section)

[{'id': 2000, 'name': '教育与教育学'}, {'id': 3000, 'name': '教育概述'}, {'id': 3001, 'name': '教育学概述'}, {'id': 2001, 'name': '教育与发展'}, {'id': 3002, 'name': '教育的产生与发展'}, {'id': 3003, 'name': '教育与人的发展'}, {'id': 3004, 'name': '教育与社会发展'}, {'id': 3005, 'name': '全民教育与终身学习'}, {'id': 2002, 'name': '教育目的'}, {'id': 3006, 'name': '教育目的的意义与性质'}, {'id': 3007, 'name': '教育目的的价值取向'}, {'id': 3008, 'name': '中国教育目的的发展'}, {'id': 2003, 'name': '教育制度与教育政策'}, {'id': 3009, 'name': '教育制度概述'}, {'id': 3010, 'name': '中国教育制度'}, {'id': 3011, 'name': '中国教育政策'}, {'id': 2004, 'name': '学生及其发展'}, {'id': 3012, 'name': '学生'}, {'id': 3013, 'name': '学生发展与教育'}, {'id': 2005, 'name': '教师'}, {'id': 3014, 'name': '教师职业概述'}, {'id': 3015, 'name': '教师的专业素养与专业发展'}, {'id': 3016, 'name': '教师学习共同体与教师发展'}, {'id': 2006, 'name': '课程与课程开发'}, {'id': 3017, 'name': '课程概述'}, {'id': 3018, 'name': '课程开发'}, {'id': 2007, 'name': '课程与教学实施'}, {'id': 3019, 'name': '课程实施'}, {'id': 3020, 'name': '教学过程'}, {'id': 3021, 'name': '教学原则'}, {'id': 3022, 'name': '教学方法'}

In [66]:
params_list = df.to_dict(orient="records")

# Cypher 批量插入查询
query = """
UNWIND $batch AS row
MERGE (n:`章`{id: row.id})
SET n.num = row.chapter_or_section, n.title = row.name
"""

# 执行查询
graph.query(query, params={"batch": params_list})
# print("数据已保存到 Neo4j 图谱")

[]

In [67]:
query = """
UNWIND $batch AS row
MERGE (n:`节`{id: row.id})
SET n.num = row.chapter_or_section, n.title = row.name
"""

# 执行查询
graph.query(query, params={"batch": params_list})
# print("数据已保存到 Neo4j 图谱")

[]

In [68]:
# for index, row in df.iterrows():
#     # 处理每一行数据
#     chapter_or_section = row['chapter_or_section']
#     name_part = row['name']
#     print(f"ID: {row['id']}, Chapter/Section: {chapter_or_section}, Name: {name_part}")

ID: 2000, Chapter/Section: 1, Name: 教育与教育学
ID: 3000, Chapter/Section: 1, Name: 教育概述
ID: 3001, Chapter/Section: 2, Name: 教育学概述
ID: 2001, Chapter/Section: 2, Name: 教育与发展
ID: 3002, Chapter/Section: 1, Name: 教育的产生与发展
ID: 3003, Chapter/Section: 2, Name: 教育与人的发展
ID: 3004, Chapter/Section: 3, Name: 教育与社会发展
ID: 3005, Chapter/Section: 4, Name: 全民教育与终身学习
ID: 2002, Chapter/Section: 3, Name: 教育目的
ID: 3006, Chapter/Section: 1, Name: 教育目的的意义与性质
ID: 3007, Chapter/Section: 2, Name: 教育目的的价值取向
ID: 3008, Chapter/Section: 3, Name: 中国教育目的的发展
ID: 2003, Chapter/Section: 4, Name: 教育制度与教育政策
ID: 3009, Chapter/Section: 1, Name: 教育制度概述
ID: 3010, Chapter/Section: 2, Name: 中国教育制度
ID: 3011, Chapter/Section: 3, Name: 中国教育政策
ID: 2004, Chapter/Section: 5, Name: 学生及其发展
ID: 3012, Chapter/Section: 1, Name: 学生
ID: 3013, Chapter/Section: 2, Name: 学生发展与教育
ID: 2005, Chapter/Section: 6, Name: 教师
ID: 3014, Chapter/Section: 1, Name: 教师职业概述
ID: 3015, Chapter/Section: 2, Name: 教师的专业素养与专业发展
ID: 3016, Chapter/Section: 3, Name: 教师学习共

In [80]:
# 给 知识点编号 获取列表

# 分配 ID（只运行一次）
# assign_id_query = """
# MATCH (kp:`知识点`)
# WITH kp ORDER BY id(kp) // 先按 Neo4j 内部 ID 排序
# WITH collect(kp) AS kps  // 收集所有知识点结点
# UNWIND range(0, size(kps) - 1) AS i
# WITH kps[i] AS kp, 4000 + i AS newId
# SET kp.id = newId
# """
# graph.query(assign_id_query)

# 获取 id 和 name
fetch_query = """
MATCH (kp:`知识点`)
RETURN kp.id AS id, kp.title AS name, kp.oid AS oid
ORDER BY kp.id
"""
point_result = graph.query(fetch_query)
print(point_result)

[{'id': 4000, 'name': '语言与思维', 'oid': '367611902371532800_wx'}, {'id': 4001, 'name': '教育与教育学', 'oid': '367611902543499264_wx'}, {'id': 4002, 'name': '教育概述', 'oid': '367611176748560384_wx'}, {'id': 4003, 'name': '情绪情感与意志', 'oid': '367611902392504320_wx'}, {'id': 4004, 'name': '教育学概述', 'oid': '367611176748560385_wx'}, {'id': 4005, 'name': '需要与动机', 'oid': '367611902413475840_wx'}, {'id': 4006, 'name': '教育与发展', 'oid': '367611902589636608_wx'}, {'id': 4007, 'name': '能力', 'oid': '367611902434447360_wx'}, {'id': 4008, 'name': '教育的产生与发展', 'oid': '367611176748560386_wx'}, {'id': 4009, 'name': '人格', 'oid': '367611902451224576_wx'}, {'id': 4010, 'name': '教育与人的发展', 'oid': '367611176748560387_wx'}, {'id': 4011, 'name': '心理发展与认知差异', 'oid': '367611902472196096_wx'}, {'id': 4012, 'name': '教育与社会发展', 'oid': '367611176748560388_wx'}, {'id': 4013, 'name': '学习心理', 'oid': '367611902493167616_wx'}, {'id': 4014, 'name': '全民教育与终身学习', 'oid': '367611176748560389_wx'}, {'id': 4015, 'name': '社会心理', 'oid': '3676119

In [ ]:
# point_result = pd.DataFrame(point_result)
# point_result.to_csv("point_result.csv", index=False, encoding="utf-8-sig")

In [72]:
# 获取知识概念和章、节列表
# 获取知识概念、章、节结点的列表
fetch_query = """
MATCH (n)
WHERE n:`主要概念`
RETURN n.id AS id, n.name AS name, n.description AS desc, n.page_content AS content, n.level AS level
ORDER BY n.id
"""
kg_result = graph.query(fetch_query)

print(kg_result)

[{'id': 1, 'name': '制度的概念', 'desc': '制度的概念包括三层含义：宏观的社会形态制度、中观的具体社会制度和微观的行为规范制度，旨在满足人们生存与发展的需要。', 'content': '\n关于制度的内涵中外学者各抒己见，有多种说法。综合中西方关于制度的认识，我们认为制度指为了满足人们的生存与发展的需要、在基于权利和义务的基础上形成的社会关系，以及与此相关的社会活动的规范系统。\n制度实际上包含三层含义：\n①指社会形态意义上的制度，这是对制度的宏观分析。如奴隶制度、封建制度、资本主义制度等，以整个社会作为实体，在区分社会发展类型、性质和阶段时使用。\n②指社会中一些具体制度，这是对社会制度的中观分析。如一个社会内部的家庭、经济、政治、教育、宗教制度等，它以具体的社会组织机构、制度设施作为实体，常在分析不同的社会关系、研究不同社会生活领域的具体问题时使用。\n③指社会规范、规则意义上的制度，是对制度的微观分析。如教育制度内部的考试制度、学习制度等，它以具体社会单位的行为规范、行为模式为实体，常在讨论个体行为时使用。', 'level': 0}, {'id': 2, 'name': '教育制度的定义与构成要素', 'desc': '', 'content': '', 'level': 0}, {'id': 3, 'name': '教育制度的定义', 'desc': '\n教育制度是一个应用十分广泛的概念，研究者给出的定义也丰富多彩。', 'content': '\n教育制度是一个应用十分广泛的概念，研究者给出的定义也丰富多彩。', 'level': 1}, {'id': 4, 'name': '《教育大辞典》的界定是“一个国家各种教育机构的体系，包括学校制度和管理学校的教育行政机构体系”', 'desc': '', 'content': '', 'level': 3}, {'id': 5, 'name': '《中国大百科全书》教育卷对教育制度作出了两种阐释', 'desc': '', 'content': '', 'level': 3}, {'id': 6, 'name': '在大陆和台湾学者联合编写的《教育百科辞典》中，教育制度有两种解释', 'desc': '', 'content': '', 'level': 3}, {'id':

In [7]:
len(kg_result)  # 4000

1415

In [79]:
kg_DF = pd.DataFrame(kg_result)
kg_DF = kg_DF[kg_DF['level'].notnull()]
print(len(kg_DF))  # 4000

kg_DF = kg_DF[['id', 'name']]

kg_ = kg_DF.to_dict(orient='records')
# print(kg_)
# print(len(kg_))  # 4000
print(len(chapter_or_section))
for item in kg_:
    chapter_or_section.append(item)
print(len(chapter_or_section))  # 4000

1368
47
1415


In [8]:
print (77 * 1415)

108955


In [81]:
import hanlp
hanlp.pretrained.sts.ALL
sts = hanlp.load(hanlp.pretrained.sts.STS_ELECTRA_BASE_ZH)

# 为 list_A 中的每个结点找到 list_B 中最相似的结点
matches = []
for node_A in point_result:
    tmp1 = []
    tmp2 = []
    for node_B in chapter_or_section:
        tmp1.append((node_A['name'], node_B['name']))
        tmp2.append((node_B['name'], node_A['name']))
    # 计算相似度
    similarity1 = sts(tmp1)
    similarity2 = sts(tmp2)
    # 计算加权平均相似度
    similarity = [0.5 * sim1 + 0.5 * sim2 for sim1, sim2 in zip(similarity1, similarity2)]
    # 取出相似度最高的结点
    max_similarity = max(similarity)
    max_index = similarity.index(max_similarity)
    # 获取对应的结点信息
    best_match = chapter_or_section[max_index]
    # 添加到结果列表中
    matches.append({
        "A_id": node_A['id'],
        "A_name": node_A['name'],
        "B_id": best_match['id'],
        "B_name": best_match['name'],
        "similarity": max_similarity
    })
    # print(f"匹配: {node_A['name']} ⟶ {best_match['name']}, 相似度: {max_similarity:.4f}")
    with open('matches.txt', 'a', encoding='utf-8') as f:
        f.write(f"匹配: {node_A['name']} ⟶ {best_match['name']}, 相似度: {max_similarity:.4f}\n")

# 按相似度排序
matches.sort(key=lambda x: x["similarity"], reverse=True)

# 输出匹配结果
for match in matches:
    print(f'[{match["A_name"]}] (ID: {match["A_id"]}) ⟶ [{match["B_name"]}] (ID: {match["B_id"]}), 相似度: {match["similarity"]:.4f}')

[教育目的] (ID: 4016) ⟶ [教育目的] (ID: 2002), 相似度: 0.9885
[德育方法与途径] (ID: 4054) ⟶ [德育方法与途径] (ID: 3028), 相似度: 0.9884
[德育过程] (ID: 4052) ⟶ [德育过程] (ID: 3027), 相似度: 0.9882
[教师] (ID: 4028) ⟶ [教师] (ID: 2005), 相似度: 0.9875
[教学方法] (ID: 4040) ⟶ [教学方法] (ID: 3022), 相似度: 0.9875
[德育概述] (ID: 4048) ⟶ [德育概述] (ID: 3025), 相似度: 0.9875
[教学过程] (ID: 4037) ⟶ [教学过程] (ID: 3020), 相似度: 0.9873
[德育的内容与原则] (ID: 4050) ⟶ [德育的内容与原则] (ID: 3026), 相似度: 0.9873
[德育] (ID: 4049) ⟶ [德育] (ID: 1365), 相似度: 0.9872
[学校德育] (ID: 4046) ⟶ [学校德育] (ID: 2008), 相似度: 0.9872
[德育模式] (ID: 4056) ⟶ [德育模式] (ID: 3029), 相似度: 0.9871
[教育与发展] (ID: 4006) ⟶ [教育与发展] (ID: 2001), 相似度: 0.9868
[教学原则] (ID: 4038) ⟶ [教学原则] (ID: 3021), 相似度: 0.9867
[教育制度概述] (ID: 4022) ⟶ [教育制度概述] (ID: 3009), 相似度: 0.9867
[学生] (ID: 4026) ⟶ [学生] (ID: 3012), 相似度: 0.9867
[教学设计] (ID: 4042) ⟶ [教学设计] (ID: 3023), 相似度: 0.9866
[教师的专业素养与专业发展] (ID: 4030) ⟶ [教师的专业素养与专业发展] (ID: 3015), 相似度: 0.9864
[中国教育制度] (ID: 4023) ⟶ [中国教育制度] (ID: 3010), 相似度: 0.9860
[教育概述] (ID: 4002) ⟶ [教育概述] (ID: 3000), 相似度: 0.9860
[教师

In [84]:
match_batch = []

for match in matches:
    if match["similarity"] > 0.7:
        # 添加到批量插入列表中
        match_batch.append({
            "A_id": match["A_id"],
            "B_id": match["B_id"],
            "similarity": match["similarity"]
        })
match_batch

[{'A_id': 4016, 'B_id': 2002, 'similarity': 0.9885131120681763},
 {'A_id': 4054, 'B_id': 3028, 'similarity': 0.9883967638015747},
 {'A_id': 4052, 'B_id': 3027, 'similarity': 0.988247275352478},
 {'A_id': 4028, 'B_id': 2005, 'similarity': 0.9875260591506958},
 {'A_id': 4040, 'B_id': 3022, 'similarity': 0.9874585866928101},
 {'A_id': 4048, 'B_id': 3025, 'similarity': 0.9874545335769653},
 {'A_id': 4037, 'B_id': 3020, 'similarity': 0.987320065498352},
 {'A_id': 4050, 'B_id': 3026, 'similarity': 0.9872654676437378},
 {'A_id': 4049, 'B_id': 1365, 'similarity': 0.9871865510940552},
 {'A_id': 4046, 'B_id': 2008, 'similarity': 0.9871677160263062},
 {'A_id': 4056, 'B_id': 3029, 'similarity': 0.9871289730072021},
 {'A_id': 4006, 'B_id': 2001, 'similarity': 0.9867604970932007},
 {'A_id': 4038, 'B_id': 3021, 'similarity': 0.9866975545883179},
 {'A_id': 4022, 'B_id': 3009, 'similarity': 0.98667311668396},
 {'A_id': 4026, 'B_id': 3012, 'similarity': 0.9866684675216675},
 {'A_id': 4042, 'B_id': 3023,

In [85]:
query = """
UNWIND $batch AS row
MATCH (a:`知识点` {id: row.A_id}), (b:`章` {id: row.B_id})
MERGE (a)-[r:对应]->(b)
SET r.similarity = row.similarity
"""
graph.query(query, params={"batch": match_batch})
query = """
UNWIND $batch AS row
MATCH (a:`知识点` {id: row.A_id}), (b:`主要概念` {id: row.B_id})
MERGE (a)-[r:对应]->(b)
SET r.similarity = row.similarity
"""
graph.query(query, params={"batch": match_batch})
query = """
UNWIND $batch AS row
MATCH (a:`知识点` {id: row.A_id}), (b:`节` {id: row.B_id})
MERGE (a)-[r:对应]->(b)
SET r.similarity = row.similarity
"""
graph.query(query, params={"batch": match_batch})

[]

In [25]:
sts([('教育概述', '教育学概述'),
     ('教育学概述', '教育概述'),
     ('教育概述', '教育概述'),
     ('教育学概述', '教育学概述')])  # 0.72

[0.9822455644607544,
 0.9858213663101196,
 0.9860208034515381,
 0.9846147298812866]